# Assignment 5: Improving the Pipeline - Updated

## Part 3: Report

Compare the performance of the different classifiers across all the metrics. Which classifier does better on which metrics? How do the results change over time? What would be your recommendation to someone who's working on this model to identify 5% of posted projects to intervene with, which model should they decide to go forward with and deploy?

### Methodology
The goal of this analysis is to predict if a project on donorschoose will not get funded within 60 days of posting. Note that based on this definition, 'positive' in machine learning terms means that a project is NOT fully funded within 60 days, and 'negative' means that a project IS fully funded within 60 days. 

To make this prediction, all of the meaningful variables available in the original data were used across all of the models: 
- school_city
- school_state
- school_metro 
- school_district
- school_county
- school_charter: whether a public charter school or not (no private schools in the dataset)
- school_magnet: whether a public magnet school or not
- teacher_prefix: teacher's gender
- primary_focus_subject: main subject for which project materials are intended
- primary_focus_area: secondary subject area
- resource_type: main type of resources requested by a project
- poverty_level: school's poverty level
- grade_level: grade level for which project materials are intended
- total_price_including_optional_support
- students_reached: number of students impacted by a project
- eligible_double_your_impact_match: whether the project was eligible for a 50% off offer by a corporate partner

The data available to build these models involves projects spanning from Jan 1, 2012 to Dec 21 2013. It is assumed here that all projects were fully funded eventually, so the question is whether they were funded within or beyond 60 days of being posted. To evaluate this, models across three validation sets spanning rolling windows of 6-months are created: 
- Split 1: July 1, 2012 - Jan 1, 2013
- Split 2: Jan 1, 2013 - July 1, 2013
- Split 3: July 1, 2013 - Jan 1, 2014 

In all three cases, the models are trained using all of the available data before that validation set after leaving a 60-day gap between the training and testing data to account for the prediction horizon. 

In a machine learning context, this is a supervised classification task where the target variable is whether a project is not funded within 60 days (not_funded_within_60days). A variety of classifiers are developed, each using the set of features listed above, across the three temporal splits, and with the classifier's parameters varied: 
- Logistic Regression
- K-Nearest Neighbor 
- Decision Tree
- Support Vector Machine
- Random Forest
- Gradient Boosting
- AdaBoost 
- Naive Bayes
- Extra Trees 

To compare these models, a variety of evaluation metrics are considered: 
- Accuracy: What proportion of predictions did the model get right? 
- Precision: What proportion of positive predictions were actually correct? 
- Recall: What proportion of actual positives were correctly predicted? 
- F1: A weighted average of precision and recall 
- AUC ROC: A measure of how well the model can distinguish between outcomes 

These metrics are considered across various percentage thresholds for converting a 'score' that each classifier predicts for a project into a binary categorization indicating whether that project should be predicted to not be funded within 60 days: 1%, 2%, 5%, 10%, 20%, 30%, and 50%. For example, at a threshold of 1%, the top 1% of projects predicted to have the highest score are predicted to not be funded within 60 days (while the remaining 99% of projects are predicted to be funded within 60 days), and the evaluation metrics are computed based on this labeling. 



### Which classifier does better on which metrics?

#### Accuracy
All of the classifiers performed fairly similarly across accuracy. On average (across temporal splits, model parameters, and evaluation thresholds), AdaBoost, Gradient Boosting, Logistic Regression, Decision Trees, and SVM performed marginally better than the others.  

#### Precision
The Logistic Regession and AdaBoost classifiers outperformed the others in their average precision, while the Naive Bayes, K-Nearest Neighbor, and Extra Trees performed particularly poorly on this metric.  

#### Recall 
Across the classifiers, the Gradient Boosting, AdaBoost, and SVM achieve the highest recall across nearly all of the thresholds, while the Extra Trees, K-Nearest Neighbor, and Naive Bayes saw the lowest recall. 

#### F1 
F1 captures the tradeoff between precision and recall using a weighted average of the two metrics. Thus, unsurprisingly, the classifiers with the highest average precision and recall (AdaBoost, Logistic Regression, Gradient Boosting, and SVM) also had the highest F1 scores. 

#### AUC ROC 
AUC ROC (or the area under the receiver operating curve) was highest on average for the AdaBoost, Gradient Boosting, and Decision Trees performing the best on average. 

### How do the results change over time?
As discussed above, three 6-month validation sets were considered for each of the models and across each threshold. The results were generally consistent across these tests sets with only minor differences over time. Specifically, the models were slightly less accurate on average in predicting the second test set. The models also had slightly lower precision and slightly lower recall in predicting the first set (therefore, also slightly lower F1 scores on this set). Notably, these trends refect general trends and averages– the performance across time also varied across particular classifiers, parameters, and thresholds with no consistent trends. 

### What would be your recommendation to someone who's working on this model to identify 5% of posted projects to intervene with– which model should they decide to go forward with and deploy?

Importantly, deciding which model to deploy should be guided by the priorities of the implementer. Specifically, a series of questions needs to be considered, for example:  
- Is it preferred to intervene in those projects that are the least likely to be funded within 60 days, or those on the cusp of being funded within 60 days? 
- Is it preferred to potentially intervene in a project that would be funded within 60 days without the intervention, or to potentially fail to intervene in a project that would not be funded within 60 days? (i.e., how should the precision-recall tradeoff be evaluated)
- Is the ultimate goal of the intervention simply to get more projects funded within 60 days, or is the goal more nuanced (e.g., to reduce the average length of time for full funding across all projects, to increase the total amount of funding for all projects, etc.) 
- Should considerations around equity be weighted (e.g., should projects in higher-poverty neighborhoods be prioritized over others, etc.) 

Given limited resources, precision is likely a key metric to consider. Across the full dataset, over 70% of projects were funded within 60 days (without intervention), and intervening in one of these projects would be an inefficient use of resources. Thus, high precision means that of the projects predicted to not be funded within 60 days, a high proportion of them were indeed not funded within 60 days (i.e., interventions would be less likely to 'mistakenly' go to projects that would've succeeded without the intervention). 

If 5% of projects can receive intervention, I would recommend the SVM or Gradient Boosting classifiers, both of which have precision above 50% for the second and third validation sets at the 5% threshold (and generally perform well across the various metrics at this threshold). Again, however, emphasizing the importance of considering the policy priorities is extremely important in this context. 